<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Computaci%C3%B3n%20cu%C3%A1ntica%20adiab%C3%A1tica%20y%20Quantum%20Annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computación cuántica adiabática y Quantum Annealing**

### Contenidos

1. [Computación cuántica adiabática](#adiabatica)
1. [Quantum Annealing](#annealing)
1. [Variational Quantum Eigensolvers (VQE)](#vqe)


<a name="adiabatica"></a>

## **Computación cuántica adiabática**

Forma alternativa de computación cuántica basada en la evolución temporal de un Hamiltoniano y en el [teorema adiabático](https://en.wikipedia.org/wiki/Adiabatic_theorem)

#### Ecuación de Schrödinger

La evolución temporal de un sistema cuántico viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

$H(t)$ es el Hamiltoniano del sistema en el instante $t$ y define la energía del mismo.

#### Teorema adiabático
Si un sistema cuántico está en un estado base (estado de menor energía o _ground state_) de un Hamiltoniano inicial y se le hace evolucionar muy lentamente hacia un Hamiltoniano final, este estado inicial evolucionará al estado base del Hamiltoniano final, tras un tiempo suficientemente largo.

Supongamos que tenemos un Hamiltoniano $H_c$ (_Hamiltoniano de coste_) cuyo estado base codifica la solución de nuestro problema.

Partimos de un Hamiltoniano $H_0$ cuyo estado base es facil de preparar y queremos llegar, en un tiempo $T$, al estado base de $H_c$. Hacemos evolucionar el sistema desde $t=0$ hasta $t=T$ como sigue:

$$H(t) = A(t)H_0 + B(t)H_c$$

donde las funciones $A$ y $B$ verifican: $A(0) = B(T) = 1$ y $A(T) = B(0) = 0$.

Un ejemplo de estas funciones son

$$
A(t) =  \left(1-\frac{t}{T}\right)\\
B(t) = \frac{t}{T}
$$


Si $t$ cambia muy lentamente, el sistema pasa de $H(0) = H_0$ a $H(T) = H_c$ manteniéndose en el estado base.

Midiendo el estado del sistema en $t=T$ obtenemos el estado $|\psi\rangle$ que minimiza $\langle\psi|H_c|\psi\rangle$ y que es la solución del problema.

En [Aharonov et al. 2007](https://doi.org/10.1137/080734479) se demostró que la computación cuántica adiabática es equivalente a la computación cuántica basada en puertas.

#### Tiempo de evolución

El teorema adiabático establece que para que $H(t)$ se mantenga en el estado base, $T$ tiene que ser inversamente proporcional al cuadrado del *spectral gap* (minima diferencia de energía entre el estado base y el primer estado excitado de $H(t)$ con $t\in[0,T]$).

Encontrar el valor del *spectral gap* es muy complejo. Además, $T$ podría llegar a ser muy grande. El *Quantum annealing* es una implementación práctica de la computación cuántica adiabática.




<a name="annealing"></a>

## **Quantum annealing**

Es un modelo similar a la computación cuántica adiabática, con ciertas limitaciones:

- El Hamiltoniano final tiene que ser un Hamiltoniano Ising:  

$$
H_c = -\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - \sum_{i=0}^{n-1} h_{i}Z_i
$$

- No se mantiene la evolución adiabática $\implies$ el estado final podría no ser el estado base de $H_c$

**Estado inicial $H_0$**

Se suele usar como estado inicial el siguiente:

$$
H_0 = -\sum_{i=0}^{n-1} X_i
$$

Este estado se denomina _mixing Hamiltonian_ y su estado base es $|\psi_0\rangle = |+\rangle^{\otimes n}$

**Evolución del Hamiltoniano**

La evolución viene dada por:

$$
H(t) = A(t)H_0 + B(t)H_c = -A(t)\sum_{i=0}^{n-1} X_i - B(t)\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - B(t)\sum_{i=0}^{n-1} h_{i}Z_i
$$


### Quantum annealers
Computadores cuánticos que resuelven problemas de optimización mediante Quantum annealing.

- [D-Wave Systems](https://www.dwavesys.com/): Primera empresa en vender quantum annealers
  - El último modelo dispone de más de 7000 cúbits
  - Suite Ocean: incluye diferentes paquetes, entre otros:
    - dwave-system: para enviar problemas a los sistemas cuánticos de D-Wave (incluyendo a través del servicio en la nube [Leap](https://cloud.dwavesys.com/)).
    - dimod: para definir estructuras de datos y modelos matemáticos de diferentes tipos de problemas.
    - dwave-networkx: integración con NetworkX para grafos.
    - dwave-neal: simulador clásico basado en [simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing).
    
  - Tipo de problemas: optimización de portfolios, optimización de rutas y gestión de flotas, optimización de líneas de producción, búsqueda de similaridad entre moléculas, etc.




In [8]:
!pip install dwave-ocean-sdk

D-Wave Cloud Client, version 0.13.6
Using the simplified configuration flow.
Try 'dwave config create --full' for more options.

Creating new configuration file: /root/.config/dwave/dwave.conf
Updating existing profile: defaults
Solver API token [skip]: 454534545
Configuration saved.


Para obtener la versión instalada:

In [9]:
!dwave --version

D-Wave Cloud Client, version 0.13.6


DWave ofrecía un acceso gratuito general (limitado) a sus sistemas, pero lo canceló en enedro de 2025:

  - El programa [Leap Quantum LaunchPad](https://www.dwavequantum.com/quantum-launchpad/) proporciona acceso por 3 meses a usuarios cualificados.

Al darse de alta, se obtiene una API key que se configura ejecutando:

In [10]:
!dwave config create

Using the simplified configuration flow.
Try 'dwave config create --full' for more options.

Updating existing configuration file: /root/.config/dwave/dwave.conf
Available profiles: defaults
Updating existing profile: defaults
Solver API token [454534545]: wdwerwrerer
Configuration saved.


**Ejemplo**

<center><img src="https://drive.google.com/uc?export=view&id=1pHIFMBSBWWciVhG1ll_SAKibigkHOiKz" alt="Malla Ising 3 nodos" width="350"  /></center>

El Hamiltoniano Ising de esta malla es:
$$
H = -\sum_{(i,j) \in E} J_{ij} Z_iZ_j - \sum_j h_j Z_j = Z_0Z_1 - 3Z_1Z_2
$$

Es decir, $J_{01}=-1$, $J_{12} = 3$, $h_j=0$.

Este problema se puede expresar usando [dimod](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/), la biblioteca base de D-Wave Ocean para representar y resolver distintos modelos de problemas con y sin restricciones, como los siguientes:

 - Modelos cuadráticos binarios (QBM), como QUBO o Ising.
 - Modelos cuadráticos con restricciones
 - Modelos discretos cuadráticos

In [12]:
from dimod import BinaryQuadraticModel, SPIN
J = {(0,1):-1, (1,2):3}
h = {}

# Expresamos el grafo simple como un modelo binario cuadrático
# SPIN: las variables toman valores {-1,1} (Ising)
problem = BinaryQuadraticModel(h, J, 0.0, SPIN)
print("El problema a resolver es:")
print(problem)

El problema a resolver es:
BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0}, {(1, 0): -1.0, (2, 1): 3.0}, 0.0, 'SPIN')


DWave ofrecía un acceso gratuito general (limitado) a sus sistemas, pero lo canceló en 2025:

  - El programa [Leap Quantum LaunchPad](https://www.dwavequantum.com/quantum-launchpad/) proporciona acceso por 3 meses a usuarios cualificados.

Para resolver el problema en un sistema de DWave, haríamos:

In [6]:
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler())

# Hacemos 10 ejecuciones
result = sampler.sample(problem, num_reads=10)
print("Las soluciones obtenidas son")
print(result)

ValueError: API token not defined

Principales funciones:

  - [DWaveSampler](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/samplers.html#dwavesampler): permite usar un sistema de DWave como un sampler en el código.
  - [EmbeddingComposite](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/composites.html#dwave.system.composites.EmbeddingComposite): permite mapear el problema en la configuraciñón física del sistema DWave.

### Solución con dimod y simulación

Dimod incluye un resolutor por fuerza bruta, que prueba todas las posibles combinaciones

In [18]:
from dimod import ExactSolver
solver = ExactSolver()
solution = solver.sample(problem)
print(solution)

   0  1  2 energy num_oc.
2 +1 +1 -1   -4.0       1
7 -1 -1 +1   -4.0       1
3 -1 +1 -1   -2.0       1
6 +1 -1 +1   -2.0       1
0 -1 -1 -1    2.0       1
5 +1 +1 +1    2.0       1
1 +1 -1 -1    4.0       1
4 -1 +1 +1    4.0       1
['SPIN', 8 rows, 8 samples, 3 variables]


Podemos usar el simulador basado en annealing simulado

In [57]:
# neal: simulador de annealing de DWave
from neal import SimulatedAnnealingSampler

# Usamos un simulador clásico en vez del D-WaveSampler
sampler = SimulatedAnnealingSampler()

# Ejecutamos 10 repeticiones
result = sampler.sample(problem, num_reads=10)

print("\nLas soluciones obtenidas son:")
print(result)



Las soluciones obtenidas son:
   0  1  2 energy num_oc.
0 -1 -1 +1   -4.0       1
1 -1 -1 +1   -4.0       1
2 +1 +1 -1   -4.0       1
3 +1 +1 -1   -4.0       1
4 -1 -1 +1   -4.0       1
5 +1 +1 -1   -4.0       1
6 +1 +1 -1   -4.0       1
7 +1 +1 -1   -4.0       1
8 -1 -1 +1   -4.0       1
9 +1 +1 -1   -4.0       1
['SPIN', 10 rows, 10 samples, 3 variables]


## **Ejercicio 3**

Para el grafo de la imagen, expresa el algoritmo MAXCUT en formato QUBO, exprésalo en dimod y resuelvelo de forma exacta y con Annealing simulado

<center><img src="https://drive.google.com/uc?export=view&id=1t37PWgnsVWRkX16wcbmFSPZNZ1D0dUOS" alt="Ejemplo MAX-CUT" width="700"  /></center>


**Solución**


Vimos que el MAXCUT expresado como QUBO equivale a minimizar $f(x)$:

$$
f(\mathbf{x}) = -\sum_{(i,j) \in E} w_{ij} (x_i + x_j - 2 x_i x_j)
$$

Para el grafo problema tenemos:

$$
f(\mathbf{x}) = -3(x_0+x_1-2x_0x_1)-2(x_1+x_2-2x_1x_2)-4(x_1+x_3-2x_1x_3)-7(x_2+x_3-2x_2x_3)-1(x_2+x_4-2x_2x_4)-5(x_3+x_4-2x_3x_4) = 6x_0x_1+4x_1x_2+8x_1x_3+14x_2x_3+2x_2x_4+10x_3x_4-3x_0-9x_1-10x_2-16x_3-6x_4
$$

In [54]:
import dimod

# Diccionario QUBO: (i,j): coeficiente
Q = {
    # Términos cuadráticos
    (0, 1): 6,   # x0 x1
    (1, 2): 4,   # x1 x2
    (1, 3): 8,   # x1 x3
    (2, 3): 14,  # x2 x3
    (2, 4): 2,   # x2 x4
    (3, 4): 10,  # x3 x4
    # Terminos lineales
    (0, 0): -3,  # x0
    (1, 1): -9,  # x1
    (2, 2): -10, # x2
    (3, 3): -16, # x3
    (4, 4): -6   # x4
}

# Crear el BinaryQuadraticModel desde QUBO
bqm = dimod.BinaryQuadraticModel.from_qubo(Q)

print(bqm)


BinaryQuadraticModel({0: -3.0, 1: -9.0, 2: -10.0, 3: -16.0, 4: -6.0}, {(1, 0): 6.0, (2, 1): 4.0, (3, 1): 8.0, (3, 2): 14.0, (4, 2): 2.0, (4, 3): 10.0}, 0.0, 'BINARY')


Resolverlo por fuerza bruta:

In [55]:
from dimod import ExactSolver
solver = ExactSolver()
solution = solver.sample(bqm)
print(solution)

    0  1  2  3  4 energy num_oc.
14  1  0  0  1  0  -19.0       1
27  0  1  1  0  1  -19.0       1
12  0  1  0  1  0  -17.0       1
25  1  0  1  0  1  -17.0       1
15  0  0  0  1  0  -16.0       1
26  1  1  1  0  1  -16.0       1
4   0  1  1  0  0  -15.0       1
9   1  0  1  1  0  -15.0       1
17  1  0  0  1  1  -15.0       1
28  0  1  0  0  1  -15.0       1
13  1  1  0  1  0  -14.0       1
24  0  0  1  0  1  -14.0       1
6   1  0  1  0  0  -13.0       1
19  0  1  0  1  1  -13.0       1
5   1  1  1  0  0  -12.0       1
8   0  0  1  1  0  -12.0       1
16  0  0  0  1  1  -12.0       1
29  1  1  0  0  1  -12.0       1
7   0  0  1  0  0  -10.0       1
18  1  1  0  1  1  -10.0       1
3   0  1  0  0  0   -9.0       1
11  0  1  1  1  0   -9.0       1
22  1  0  1  1  1   -9.0       1
30  1  0  0  0  1   -9.0       1
2   1  1  0  0  0   -6.0       1
10  1  1  1  1  0   -6.0       1
23  0  0  1  1  1   -6.0       1
31  0  0  0  0  1   -6.0       1
1   1  0  0  0  0   -3.0       1
20  0  1  

Resolverlo usando annealing simulado

In [56]:
# neal: simulador de annealing de DWave
from neal import SimulatedAnnealingSampler

# Usamos un simulador clásico en vez del D-WaveSampler
sampler = SimulatedAnnealingSampler()

# Ejecutamos 10 repeticiones
result = sampler.sample(bqm, num_reads=10)

print("\nThe solutions that we have obtained are:")
print(result)


The solutions that we have obtained are:
   0  1  2  3  4 energy num_oc.
0  1  0  0  1  0  -19.0       1
1  0  1  1  0  1  -19.0       1
2  0  1  1  0  1  -19.0       1
3  1  0  0  1  0  -19.0       1
4  0  1  1  0  1  -19.0       1
5  0  1  1  0  1  -19.0       1
6  1  0  0  1  0  -19.0       1
7  0  1  1  0  1  -19.0       1
8  1  0  0  1  0  -19.0       1
9  1  0  0  1  0  -19.0       1
['BINARY', 10 rows, 10 samples, 5 variables]
